## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/LaboratoryOfComputationalPhysics_Y3/data/

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1.
file_name = 'data/data_000637.txt'
data = pd.read_csv(file_name, nrows=20000)
data

In [ ]:
# 2.
x = np.max(data['BX_COUNTER'])
print('Number of BX in a ORBIT:', x)

In [ ]:
# 3 and 4.
data['TIMENS'] = data['TDC_MEAS']*25/30 + data['BX_COUNTER']*25 + (x*25)*data['ORBIT_CNT']
min_t = np.min(data['TIMENS'])
max_t = np.max(data['TIMENS'])
time_t = max_t - min_t
print('Time in milliseconds:', time_t*1e-6)

In [ ]:
# 5.
data['HEAD'] = np.random.randint( 0, 2, len(data['HEAD']))

In [ ]:
# 6.
filtered_data = data[data['HEAD'] == 1]

In [ ]:
# 7.
fpga_0 = data[data['FPGA']==0]
fpga_1 = data[data['FPGA']==1]
occ_0 = fpga_0['TDC_CHANNEL'].value_counts()
occ_1 = fpga_1['TDC_CHANNEL'].value_counts()

%matplotlib inline

plt.scatter(np.array(occ_0.index), np.array(occ_0), s=5, label='FPGA 0')
plt.scatter(np.array(occ_1.index), np.array(occ_1), s=5, label='FPGA 1')
plt.xlabel('TDC_CHANNEL')
plt.ylabel('Number of counts')
plt.legend()

In [ ]:
# 8.
occ = data.groupby('TDC_CHANNEL').count()
n_channels = 3
noisest_channels = np.zeros(n_channels)
for i in range(0, n_channels):
    noisest_channels[i] = occ['HEAD'].idxmax()
    occ = occ.drop(occ['HEAD'].idxmax())
print(noisest_channels)

In [ ]:
# 9.
n_unique_orbits = data['ORBIT_CNT'].nunique()
print('Number of unique orbits: ', n_unique_orbits)

n_unique_orbits_tdc139 = data[data['TDC_CHANNEL']==139]['ORBIT_CNT'].nunique()
print('Number of unique orbits with TDC_CHANNEL = 139:' , n_unique_orbits_tdc139)